<a href="https://colab.research.google.com/github/gulabpatel/Rapids/blob/main/01%3A%20Rapids_Dataframe_API_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rapids introductory video : https://www.youtube.com/watch?v=pEuuK1GVscM

Video walkthrough the code : https://www.youtube.com/watch?v=Efv0_qWzDYU

![alt text](https://developer.nvidia.com/sites/default/files/akamai/machine-learning/RAPIDS_workflow2.png)

In [1]:
!nvidia-smi

Wed Jun 23 09:41:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 272 (delta 56), reused 38 (delta 19), pack-reused 171
Receiving objects: 100% (272/272), 79.66 KiB | 1.35 MiB/s, done.
Resolving deltas: 100% (118/118), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:36
🔁 Restarting kernel...


In [1]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
!python rapidsai-csp-utils/colab/install_rapids.py stable

In [3]:
#only for 0.10
!conda install -y --prefix /usr/local -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge dask-cudf xgboost

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.



In [4]:
import cudf
import numpy as np
import dask_cudf

In [5]:
bank_df=cudf.read_csv('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/bank-full.csv',sep=';')

In [6]:
bank_df.shape

(45211, 17)

In [7]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [8]:
np.unique(bank_df.y)

0     no
1    yes
Name: y, dtype: object

In [9]:
!nvidia-smi

Wed Jun 23 10:13:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |    432MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1 - age (numeric) 

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur", "student","blue-collar","self-employed","retired","technician","services") 

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed) 

4 - education (categorical: "unknown","secondary","primary","tertiary") 

5 - default: has credit in default? (binary: "yes","no") 

6 - balance: average yearly balance, in euros (numeric) 

7 - housing: has housing loan? (binary: "yes","no") 

8 - loan: has personal loan? (binary: "yes","no")

- related with the last contact of the current campaign:

9 - contact: contact communication type (categorical: "unknown","telephone","cellular")

10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

- other attributes:

13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted) 

15 - previous: number of contacts performed before this campaign and for this client (numeric) 

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

- output variable (desired target):

17 - y - has the client subscribed a term deposit? (binary: "yes","no")

In [10]:
print ("Rows     : " ,bank_df.shape[0])
print ("Columns  : " ,bank_df.shape[1])

Rows     :  45211
Columns  :  17


In [11]:
bank_df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [12]:
bank_df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: uint64

In [13]:
bank_df.y.value_counts()

no     39922
yes     5289
Name: y, dtype: int32

In [14]:
import time

In [16]:
start_time = time.time()
bank_df.describe()
print("Time Taken GPU %s seconds " % (str(time.time() - start_time)))

Time Taken GPU 0.07788324356079102 seconds 


In [17]:
dcudf = dask_cudf.from_cudf(bank_df, npartitions=2) 
start_time = time.time()
dcudf.describe()
print("Time Taken GPU %s seconds " % (str(time.time() - start_time)))

Time Taken GPU 0.1418015956878662 seconds 


In [18]:
bank_df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [19]:
bank_df.groupby(['marital', 'y']).agg({'balance':'mean'})

,,balance
marital,y,
married,no,1370.746228
divorced,no,1107.095747
single,yes,1674.875523
divorced,yes,1707.964630
single,no,1235.869921
married,yes,1915.810163


In [20]:
bank_df.groupby(['marital', 'y']).agg({'balance':'mean','y':'count'})

,,balance,y
marital,y,,
married,no,1370.746228,24459
divorced,no,1107.095747,4585
single,yes,1674.875523,1912
divorced,yes,1707.964630,622
single,no,1235.869921,10878
married,yes,1915.810163,2755


In [21]:
bank_df.groupby(['loan', 'y']).agg({'balance':'mean','y':'count'})

balance      y
loan y                      
yes  no    766.481953   6760
     yes   883.642562    484
no   yes  1897.001041   4805
     no   1413.228726  33162

In [22]:
loan_outcome = bank_df.groupby(['loan', 'y']).agg({'balance':'mean','y':'count'})

In [23]:
loan_outcome['y_perct'] = loan_outcome['y']/loan_outcome['y'].sum()

In [24]:
loan_outcome

balance      y   y_perct
loan y                                
yes  no    766.481953   6760  0.149521
     yes   883.642562    484  0.010705
no   yes  1897.001041   4805  0.106279
     no   1413.228726  33162  0.733494

In [25]:
def convert_hour(duration):
    return duration/60

In [26]:
bank_df['duration_hour'] = bank_df['duration'].applymap(convert_hour)

In [27]:
bank_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,duration_hour
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,4.350000
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2.516667
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,1.266667
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,1.533333
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,3.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,16.283333
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,7.600000
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,18.783333
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,8.466667


In [28]:
bank_df.groupby('y').balance.mean()

y
no     1303.714969
yes    1804.267915
Name: balance, dtype: float64

In [29]:
bank_df.groupby('y').campaign.mean()

y
no     2.846350
yes    2.141047
Name: campaign, dtype: float64

In [30]:
bank_df['campaign'].quantile(q = [0.25,0.5, 0.75, 0.9, 0.95, 1.0])

0.25     1.0
0.50     2.0
0.75     3.0
0.90     5.0
0.95     8.0
1.00    63.0
Name: campaign, dtype: float64

In [31]:
bank_campaign_df = bank_df.query("campaign <= 8")

In [32]:
bank_campaign_df.groupby(['campaign', 'y']).agg({'y':'count'})

y
campaign y         
3        yes    618
5        no    1625
2        no   11104
6        no    1199
         yes     92
8        yes     32
1        yes   2561
         no   14983
7        no     688
         yes     47
4        no    3205
         yes    317
3        no    4903
8        no     508
5        yes    139
2        yes   1401

In [33]:
bank_df.education.value_counts()

secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: education, dtype: int32

In [34]:
bank_df.groupby(['education','y']).agg({'y':'count'})

,,y
education,y,
secondary,yes,2450
unknown,yes,252
secondary,no,20752
tertiary,no,11305
primary,no,6260
tertiary,yes,1996
unknown,no,1605
primary,yes,591
